In [ ]:
pip install aspect-based-sentiment-analysis

In [ ]:
import aspect_based_sentiment_analysis as absa

nlp = absa.load()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from google.colab import files
import numpy as np 
import pandas as pd

# Sentiment - Packaging 

In [ ]:
path2 = '/content/drive/MyDrive/5153/meta_and_review_sentiment_aspect_delivery_cs.csv'
df = pd.read_csv(path2)

In [ ]:
df.shape

(133159, 39)

In [ ]:
df.columns
#delivery customer

Index(['Unnamed: 0', 'asin', 'review_x', 'overall', 'helpful', 'summary',
       'unixReviewTime', 'reviewTime', 'reviewerID', 'title', 'brand', 'price',
       'salesRank', 'polarity_score', 'neutral_score', 'negative_score',
       'positive_score', 'neu_delivery', 'neg_delivery', 'pos_delivery',
       'neu_shipment', 'neg_shipment', 'pos_shipment', 'review_len',
       'review_y', 'Ori_index', 'customer_aspect', 'neu_return_policy',
       'neg_return_policy', 'pos_return_policy', 'neu_exchange',
       'neg_exchange', 'pos_exchange', 'neu_service', 'neg_service',
       'pos_service', 'neu_support', 'neg_support', 'pos_support'],
      dtype='object')

In [ ]:
text_df = df[['review']]
text_df['Ori_index'] = text_df.index
text_df.shape

(133159, 2)

In [ ]:
sum(text_df.review.isnull())

0

### filter reviews with package aspect

In [ ]:
package_aspect = ['package', 'packaging', 
                   'wrapping', 'pack']

In [ ]:
#check condition; more likely to be hair condition, scalp condition and so on
import re
pattern = r'\bcondition\b'
with pd.option_context ('display.max_colwidth', None):
  display (text_df[text_df.review.apply(lambda x: 1 if re.findall(pattern,x) else 0)==1].review)

In [ ]:
#check cs (for customer service aspect) #only less than 10 
import re
pattern = r'\bcs\b'
with pd.option_context ('display.max_colwidth', None):
  display (text_df[text_df.review.apply(lambda x: 1 if re.findall(pattern,x) else 0)==1].review)

In [ ]:
text_df.loc[text_df['review'].str.contains('package|packaging|wrapping|pack',regex=True), 'package_aspect'] = 1
text_df.package_aspect.fillna(0, inplace=True)

In [ ]:
sum(text_df.package_aspect==1)

4463

In [ ]:
packaing_idx = list(text_df[text_df.package_aspect==1].index)
long_text_512_idx_list = list(text_df[text_df.loc[:,'review'].str.len() >=1999].review.index)
package_index_removed = [x  for x in packaing_idx if x in long_text_512_idx_list]
len(package_index_removed)

154

In [ ]:
#update package index
#remove extremel long text which this package can not handle
packaing_idx =[x for x in packaing_idx if x not in package_index_removed]


In [ ]:
len(packaing_idx)

4309

In [ ]:
for idx in packaing_idx: 
  pack, package  = nlp(text_df.review.iloc[idx], aspects=['pack', 'package'])
  text_df.loc[idx, 'neu_pack'] = pack.scores[0]
  text_df.loc[idx,'neg_pack'] = pack.scores[1]
  text_df.loc[idx,'pos_pack'] = pack.scores[2]
  text_df.loc[idx, 'neu_package'] = package.scores[0]
  text_df.loc[idx,'neg_package'] = package.scores[1]
  text_df.loc[idx,'pos_package'] = package.scores[2]

In [ ]:
text_df.shape

(133159, 9)

In [ ]:
# impute na by zero 
text_df = text_df.fillna(0)

In [ ]:
text_df.head()

,review,Ori_index,package_aspect,neu_pack,neg_pack,pos_pack,neu_package,neg_package,pos_package
0,this was a good buy great shampoo and conditio...,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,i have used kms in the past and always loved t...,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,i remember kms products being a prestige salon...,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,this is a wonderful shampoo doesnt make your h...,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,i have a very dry frizzy hair i have tried man...,4,1.0,0.079432,0.899014,0.021554,0.379829,0.084498,0.535672


In [ ]:
text_df.to_csv('review_sentiment_aspect_package.csv', index=None)
files.download('review_sentiment_aspect_package.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
path3 = '/content/drive/MyDrive/5153/review_sentiment_aspect_package.csv'
df3=pd.read_csv(path3)
df3.columns
#package make more sense

Index(['review', 'Ori_index', 'package_aspect', 'neu_pack', 'neg_pack',
       'pos_pack', 'neu_package', 'neg_package', 'pos_package'],
      dtype='object')

In [ ]:
#check 
df3[df3.pos_package!=0][['pos_pack', 'pos_package']]

,pos_pack,pos_package
4,0.021554,0.535672
12,0.009155,0.575158
135,0.043165,0.198378
158,0.007172,0.023459
170,0.028247,0.003691
...,...,...
133078,0.706504,0.966684
133104,0.419795,0.998024
133120,0.013276,0.007332
133131,0.354877,0.992254


In [ ]:
#df3.loc[4,'review']
#df3.loc[12,['review', 'neu_pack', 'neg_pack',
#       'pos_pack', 'neu_package', 'neg_package', 'pos_package']] #love the new packagingd
df3.loc[133104, 'review']

'first thing i would would like to say is i love the brand of products they are always high quality and do exactly what they say they do as a cosmetologist and skin care expert who worked int he field for 10 years really like the results  i have thin but alot of hair it did not weigh it down let my hair smooth and easy to style  it also helped my daughters flaky dry itchy scalp  shipping was good and product was well protected and packaged arrived very fast and i am pleased with product '

In [ ]:
#merge package aspect with delivery, customer service aspect 
df_full = df.merge(df3, how = 'left', left_index=True, right_on= 'Ori_index')
df_full.shape

(133159, 49)

In [ ]:
df_full.columns

Index(['Ori_index', 'Unnamed: 0', 'asin', 'review_x', 'overall', 'helpful',
       'summary', 'unixReviewTime', 'reviewTime', 'reviewerID', 'title',
       'brand', 'price', 'salesRank', 'polarity_score', 'neutral_score',
       'negative_score', 'positive_score', 'neu_delivery', 'neg_delivery',
       'pos_delivery', 'neu_shipment', 'neg_shipment', 'pos_shipment',
       'review_len', 'review_y', 'Ori_index_x', 'customer_aspect',
       'neu_return_policy', 'neg_return_policy', 'pos_return_policy',
       'neu_exchange', 'neg_exchange', 'pos_exchange', 'neu_service',
       'neg_service', 'pos_service', 'neu_support', 'neg_support',
       'pos_support', 'review', 'Ori_index_y', 'package_aspect', 'neu_pack',
       'neg_pack', 'pos_pack', 'neu_package', 'neg_package', 'pos_package'],
      dtype='object')

In [ ]:
df_full.drop(['Unnamed: 0','customer_aspect', 'Ori_index_x', 'review_y','Ori_index_y', 'package_aspect'], axis=1, inplace=True)

In [ ]:
df_full.drop(['review'], axis=1, inplace=True)
df_full = df_full.rename(columns={'review_x':'review'})

In [ ]:
#output file
df_full.to_csv('meta_and_review_sentiment_aspect_delivery_cs_package.csv', index=None)
files.download('meta_and_review_sentiment_aspect_delivery_cs_package.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>